In [17]:
import matplotlib.pyplot as plt
import os, sys

from astropy.io import fits
from astropy.table import Table

import numpy as np

import matplotlib.colors as colors
from astropy.visualization import imshow_norm, MinMaxInterval, LogStretch,PercentileInterval, ImageNormalize

import sep
sys.path.append(os.path.expanduser('~/repos/ReipurthBallyProject/src'))
from catalog import *
import sep

(https://stackoverflow.com/questions/37031356/check-if-points-are-inside-ellipse-faster-than-contains-point-method)

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# import numpy as np

# fig,ax = plt.subplots(1)
# ax.set_aspect('equal')

# # Some test points
# x = np.random.rand(500)*0.5+0.7
# y = np.random.rand(500)*0.5+0.7

# # The ellipse
# g_ell_center = (0.8882, 0.8882)
# g_ell_width = 0.36401857095483
# g_ell_height = 0.16928136341606
# angle = 30.

# g_ellipse = patches.Ellipse(g_ell_center, g_ell_width, g_ell_height, angle=angle, fill=False, edgecolor='green', linewidth=2)
# ax.add_patch(g_ellipse)

# cos_angle = np.cos(np.radians(180.-angle))
# sin_angle = np.sin(np.radians(180.-angle))

# xc = x - g_ell_center[0]
# yc = y - g_ell_center[1]

# xct = xc * cos_angle - yc * sin_angle
# yct = xc * sin_angle + yc * cos_angle 

# rad_cc = (xct**2/(g_ell_width/2.)**2) + (yct**2/(g_ell_height/2.)**2)

# # Set the colors. Black if outside the ellipse, green if inside
# colors_array = np.array(['black'] * len(rad_cc))
# colors_array[np.where(rad_cc <= 1.)[0]] = 'green'

# ax.scatter(x,y,c=colors_array,linewidths=0.3)

# plt.show()

In [18]:
with open('/home/kevin/Documents/HH24/regions/jets.reg') as reg:
    jets = {}
    for line in reg.readlines():
        if not line.startswith('ellipse'): continue
        line = line[8:]
        jetstr = line.split('{')[-1].split('}')[0]
        paramstr = line.split('()')[-1].split(')')[0]
        params = paramstr.split(',')

        jets[jetstr]= {'center':(float(params[0]), float(params[1])), 'a': float(params[2]), 'b':float(params[3]), 'angle':float(params[4])}

In [19]:
jets['Jet C']

{'center': (3520.1664, 2890.2656),
 'a': 1494.4085,
 'b': 200.0,
 'angle': 45.477483}

In [51]:
def pts_in_ellipse(ellipse, x, y, scale=6.0):
    angle=ellipse['angle']
    a = scale*ellipse['a']/2.0
    b = scale*ellipse['b']/2.0
    center = ellipse['center']

    cos_angle = np.cos(np.radians(180.-angle))
    sin_angle = np.sin(np.radians(180.-angle))

    xc = x - center[0]
    yc = y - center[1]

    xct = xc * cos_angle - yc * sin_angle
    yct = xc * sin_angle + yc * cos_angle 

    rad_cc = (xct**2)/(a**2) + (yct**2)/(b**2)

    return rad_cc <= 1.0

In [21]:
obsdir = '/home/kevin/Documents/HH24'
frameid = '2024_03_15_F164N_reg'
with fits.open(os.path.join(obsdir,'registered', frameid+'.fits')) as f:
#with fits.open(os.path.join(obsdir,'lucy-richardson', frameid+'_lr15.fits')) as f:
    print(f'Length: {len(f)}')
    for i,hdr in enumerate(f):
        xt = hdr.header.pop('XTENSION','Unknown')
        xn = hdr.header.pop('EXTNAME', 'Unknown')
        rn = hdr.header.pop('ROOTNAME', 'Unknown')
        radesys = hdr.header.pop('RADESYS', 'Unknown')
        print(f'Index: {i}, Extension: {xt}, Rootname: {rn}, Extname: {xn}, RADESYS: {radesys}')
    hdr = f[0].header.copy()
    img = f[0].data.copy()

Length: 1
Index: 0, Extension: Unknown, Rootname: Unknown, Extname: Unknown, RADESYS: ICRS


In [23]:
from r_d_src.img_find_objects import find_stars

In [24]:
regout = os.path.join(obsdir,'knotcat', frameid+'.xml')
#if not os.path.exists(regout):
zz=find_stars(frameid, hdr, img, regout,byteswap=True, thresh=5,
            deblend_cont=0.000001,
            deblend_nthresh=16)

2024_03_15_F164N_reg: Number of objects identified: 2540


In [59]:
knotcat = load_catalog(regout)

In [60]:
knotcat.remove_column('Jet')
knotcat['Jet'] = 0
for i, j in enumerate(jets):
    inJet = pts_in_ellipse(jets[j], knotcat['fits_x'], knotcat['fits_y'], scale=2)
    knotcat['Jet'][inJet] = i+1

In [62]:
injets = knotcat['Jet'] != 0
knotcat = knotcat[injets]

In [63]:
knotcat.write(regout, format='votable',overwrite=True)

In [66]:
for i, j in enumerate(jets):
    print(f'Jet number: {i+1}, Jet Name: {j}')

Jet number: 1, Jet Name: Jet E
Jet number: 2, Jet Name: Jet G
Jet number: 3, Jet Name: Jet C
Jet number: 4, Jet Name: Jet J
Jet number: 5, Jet Name: Jet L
